In [ ]:
import pandas as pd
import numpy as np
import dash
import mysql.connector
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [ ]:
conn = mysql.connector.Connect(user='***', password='***',
                              host='***',
                              database='***')
c = conn.cursor()
c.execute("SHOW TABLES")
tables = c.fetchall()
tables

In [ ]:
query = ("SELECT * from rs_2017_roomstatinfo ")

In [ ]:
df = pd.read_sql(query, conn)
df.shape

In [ ]:
print(list(df))

In [ ]:
conn.close()

In [ ]:
df.head()

In [ ]:
# df.startTime = pd.to_datetime(df.startTime, format='%d%b%Y:%H:%M:%S.%f')

type(df.startTime[1])

In [ ]:
print(df.startTime.max())
print(df.startTime.min())

In [ ]:
df['meeting_minutes'] = (df.duration/60).round(1)
df.head()

In [ ]:
df['date'] = df.startTime.map(lambda x: x.strftime('%Y-%m'))
df.head()

In [ ]:
%%time
def f(row):
    if row['meeting_minutes'] >= 30 and row['userCount'] > 2:
        val = 1
    else:
        val = 0
    return val

df['会议是否达标'] = df.apply(f, axis=1)
display(df.head())

In [ ]:
df_gb_CustomerName = df.groupby(['CompanyName', 'date']).agg({
    'duration': [max], 
    'userCount':  [max], 
    'CompanyName': 'count',
    '会议是否达标':'sum'
})
df_gb_CustomerName.head()

In [ ]:
df_gb_CustomerName.columns = df_gb_CustomerName.columns.droplevel(level=0)
df_gb_CustomerName = df_gb_CustomerName.rename(columns={"max": "max_duration", "max": "max_userCount", "count": "meeting_count","sum": "达标会议_sum"})
df_gb_CustomerName = df_gb_CustomerName.reset_index()
df_gb_CustomerName.head(20)

In [ ]:
df_gb_CustomerName['月环比增长率'] = ((df_gb_CustomerName['meeting_count']-df_gb_CustomerName.groupby('CompanyName')['meeting_count'].shift()) / 
                                df_gb_CustomerName.groupby('CompanyName')['meeting_count'].shift()).round(2)
df_gb_CustomerName.head(20)

In [ ]:
df_gb_CustomerName.to_csv('test.csv',index=False)